# Termowizja

### 10.1 Binaryzacja

In [47]:
import numpy as np
import cv2

def connect_rectangles(stats, eta=20):
    groups = []
    used = [False] * len(stats)

    for i in range(len(stats)):
        if used[i]:
            continue
        x1, y1, w1, h1, a = stats[i]
        x_center1 = x1 + w1 // 2

        group = [i]
        used[i] = True

        for j in range(len(stats)):
            if i == j or used[j]:
                continue

            x2, y2, w2, h2, _ = stats[j]
            x_center2 = x2 + w2 // 2

            if abs(x_center1 - x_center2) < eta:
                group.append(j)
                used[j] = True

        groups.append(group)

    # Dla każdej grupy tworzymy otaczający prostokąt
    new_stats = []
    for group in groups:
        x_vals = []
        y_vals = []
        x_max_vals = []
        y_max_vals = []
        total_area = 0

        for idx in group:
            x, y, w, h, area = stats[idx]
            x_vals.append(x)
            y_vals.append(y)
            x_max_vals.append(x + w)
            y_max_vals.append(y + h)
            total_area += area

        x_min = min(x_vals)
        y_min = min(y_vals)
        x_max = max(x_max_vals)
        y_max = max(y_max_vals)

        new_stats.append([x_min, y_min, x_max - x_min, y_max - y_min, total_area])

    return np.array(new_stats)

def filter_stats(stats):
    new_stats = []
    for i in range(len(stats)):
        x, y, w, h, a = stats[i]

        if w == 0:  # Unikaj dzielenia przez zero
            continue

        aspect_ratio = float(h) / w

        # Filtrujemy tylko prostokąty z rozsądnym stosunkiem boków (np. między 1 a 2)
        if 1.5 <= aspect_ratio <= 4:
            new_stats.append(stats[i])

    return np.array(new_stats)

In [50]:
cap = cv2.VideoCapture(r'/home/plorenc/Desktop/AiR_ISS/AVS/Materiały-lab10/vid1_IR.avi')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(G, 70, 255, cv2.THRESH_BINARY)
    img = cv2.medianBlur(img, ksize=5)
    img = cv2.erode(img, kernel=np.ones((5, 5), np.uint8), iterations=1)
    img = cv2.dilate(img, kernel=(5, 5), iterations=5)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img)


    stats = stats[1:]

    stats = filter_stats(stats)

    merged_stats = connect_rectangles(stats, eta=70)

    for i in range(len(merged_stats)):
        x, y, w, h, area = merged_stats[i]

        cv2.putText(img, f"{i+1}, {h}, {w}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)

    cv2.imshow('IR', img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()